<a href="https://colab.research.google.com/github/Nick088Official/Stable_Diffusion_Finetuned_Minecraft_Skin_Generator/blob/main/Stable_Diffusion_Finetuned_Minecraft_Skin_Generator_WEB_UI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Stable Diffusion Finetuned Minecraft Skin Generator WEB UI

Based on the [Hugging Face Space](https://huggingface.co/spaces/Nick088/Stable_Diffusion_Finetuned_Minecraft_Skin_Generator)

**WARNING: WEB UI COULD RISK YOUR FREE TIER COLAB ACC**

Credits:
- Author of the AI Model: Cory Spencer <cory@monadical.com>
- Forked Improved Version & Ports: [Nick088](https://linktr.ee/Nick088)
- daroche (helping me fix the 3d model texture isue)
- [Brottweiler](https://gist.github.com/Brottweiler/483d0856c6692ef70cf90bf1a85ce364) (script to fix the 3d model texture)
- [not-holar](https://huggingface.co/not-holar) (made the rendering of the image in the web ui look pixelated like minecraft and have a checkered background)
- [meew](https://huggingface.co/spaces/meeww/Minecraft_Skin_Generator/blob/main/models/player_model.glb) (Minecraft Player 3d model)

In [ ]:
#@title Install
#@markdown If you get a restart runtime warning, you don't have to, just click cancel
!git clone https://huggingface.co/spaces/Nick088/Stable_Diffusion_Finetuned_Minecraft_Skin_Generator
%cd Stable_Diffusion_Finetuned_Minecraft_Skin_Generator
!pip install -r requirements.txt
!pip install pyngrok
!apt-get install $(cat packages.txt)
from IPython.display import clear_output
clear_output()
print("Installed!")

In [ ]:
#@title Run Inference UI
import time

#@markdown The type of tunnel you wanna use for seeing the public link, so that if one of them is down, you can use the other one.
Tunnel = "Gradio" #@param ["Gradio", "Ngrok", "Cloudfare"]

#@markdown Also when using Ngrok or Cloudfare, you need to wait for the Local URL to appear, and only after that click on the Public URL which is above.

#@markdown Use the following option only if you chose Ngrok as the Tunnel:

#@markdown You can get the Ngrok Tunnel Authtoken here: https://dashboard.ngrok.com/tunnels/authtokens/new.

ngrok_tunnel_authtoken = "" #@param {type:"string"}

!sed -i 's/import spaces/ /g'  /content/Stable_Diffusion_Finetuned_Minecraft_Skin_Generator/app.py
!sed -i 's/@spaces.GPU()/ /g'  /content/Stable_Diffusion_Finetuned_Minecraft_Skin_Generator/app.py

if Tunnel == "Gradio":
  !sed -i 's/share=False/share=True/g'  /content/Stable_Diffusion_Finetuned_Minecraft_Skin_Generator/app.py
if Tunnel == "Ngrok":
  !sed -i 's/share=True/share=False/g'  /content/Stable_Diffusion_Finetuned_Minecraft_Skin_Generator/app.py
  from pyngrok import ngrok
  ngrok.set_auth_token(ngrok_tunnel_authtoken)
  http_tunnel = ngrok.connect(7860, bind_tls=True)
  print("Ngrok Tunnel Public URL:", http_tunnel.public_url)
elif Tunnel == "Cloudfare":
  !sed -i 's/share=True/share=False/g'  /content/Stable_Diffusion_Finetuned_Minecraft_Skin_Generator/app.py
  # download cloudfare
  !curl -LO https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
  !dpkg -i cloudflared-linux-amd64.deb
  !rm -rf nohup.out
  # Run cloudflare
  !nohup cloudflared tunnel --url localhost:7860 &
  clear_output()
  time.sleep(5)
  # Find and print the Cloudflare URL with a prefix
  cloudflare_url = !grep -oE "https://[a-zA-Z0-9.-]+\.trycloudflare\.com" nohup.out
  print(f"Cloudfare Tunnel Public URL: {cloudflare_url[0]}")

!python app.py